These are using dataset from 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import mathimport warnings
warnings.filterwarnings("ignore")

PRETRAINED

In [ ]:
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model,load_model


conv_base = InceptionV3(weights = 'imagenet',include_top=False,input_shape=(300,300,3))

output = conv_base.layers[-1].output
output = keras.layers.Flatten()(output)
model_tl = Model(conv_base.input, output)

model_tl.trainable = False
for layer in model_tl.layers:
  layer.trainable = False

layers = [(layer,layer.name,layer.trainable)for layer in
          model_tl.layers]
model_layers = pd.DataFrame(layers,columns=['layer Type','Layer Name','Layer Trainable'])

print(model_layers)


In [ ]:
#preproses dan data augmentasi
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Dense,Dropout,Conv2D,MaxPool2D,Flatten

test_size = 400
batch_size =32
epochs=25

#dataset 
train_path = '/content/drive/MyDrive/TALatihan/carnivores/train'
test_path = '/content/drive/MyDrive/TALatihan/carnivores/test'

target_size = (300,300) #resize all images to 300x300

train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3,  
                                   rotation_range=50,
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   brightness_range = [0.8, 1.2],
                                   fill_mode='nearest',        
                                   validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

#the list of classes will be automatically inferred from the subdirectory names/stucture under train_dir
train_generator = train_datagen.flow_from_directory(
                  train_path,
                  target_size=target_size,#  
                  batch_size=batch_size,
                  class_mode='categorical',
                  subset='training')
validation_generator = train_datagen.flow_from_directory(
                       train_path,
                       target_size=target_size,
                       batch_size=batch_size,
                       class_mode='categorical',
                       subset='validation')
#building model arsitektur
from keras.models import Sequential
from keras.layers import Dense,Dropout,Conv2D,MaxPool2D,Flatten
from keras import optimizers
#building a linear stack of layers with the sequential model
model = Sequential()
model.add(model_tl)

#hidden layer
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
#output layer
model.add(Dense(4,activation='softmax'))
#resume training abis internet dc/mati lampu
model.load_weights('/content/drive/MyDrive/TALatihan/CP/epochs:025-val_acc:0.993.hdf5') 
#compiling the sequential model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
print(model.summary())


In [ ]:
#Saving model checkpoint
from keras.callbacks import *
#/content/drive/MyDrive/TALatihan/CP

filepath ='/content/drive/MyDrive/TALatihan/CP/epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5'
checkpoint = ModelCheckpoint(filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=False,
                             save_freq='epoch',
                             mode='max')
callbacks_list=[checkpoint]


In [ ]:

#model performance
#model evaluation
scores_train = model.evaluate(train_generator,verbose=1)
scores_validation = model.evaluate(validation_generator,verbose=1)
print("Train Accuracy: %.2f%%" % (scores_train[1]*100))
print("validation akurasi : %.2f%% "%(scores_validation[1]*100))

In [ ]:
def LearningCurve(history):
#summarize history for akurasi
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('model accurasi')
  plt.ylabel('accurasi')
  plt.xlabel('epoch')
  plt.legend(['train','validation'],loc='upper left')
plt.show()

In [ ]:

def LearningCurve(history):
#summarize history for akurasi
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('model accurasi')
  plt.ylabel('accurasi')
  plt.xlabel('epoch')
  plt.legend(['train','validation'],loc='upper left')
plt.show()

In [ ]:
#we take the ceiling because we dont drop the remainder of the batch
import math
compute_steps_per_epoch = lambda x: int(math.ceil(1. * x / batch_size))
test_steps = compute_steps_per_epoch(test_size)
test_generator = test_datagen.flow_from_directory(
                 test_path,
                 target_size=target_size, 
                 batch_size=batch_size,
                 class_mode=None,
                 shuffle=False)
test_generator.reset()